In [1]:
#!pip install -U googlemaps
#!pip install gmaps
#!pip install networkx
import networkx as nx
import math
import requests
import json
import pandas as pd
import googlemaps
import gmaps
req = requests.post('http://127.0.0.1:5007/',
                    json = {'age':20,'been2tko':5,'howlong':6,'budget':50000,
                            'isFemale':1,'with_child':0,'crampedSechedule':True}
                   )
req.encoding='utf-8'
req = req.json()
print(len(req),type(req),req)
day2=pd.read_json(req[1])
print(day2)
# print(req.content.decode('utf-8'))

5 <class 'list'> ['[{"index":264,"name":"東京迪士尼樂園 Tokyo Disneyland","rank":"排名第 2的Maihama觀光 (共5家)","address":"千葉縣浦安Maihama1-1279-8511","child":1.0,"latitude":35.632906,"longitude":139.880391,"score":94,"chosen":true}]', '[{"index":226,"name":" Nittere Plaza","rank":"排名第 50的港區觀光 (共626家)","address":"東京都港區1-6-1 HigashishinbashiNippon Television Tower105-0021","child":0.233813924,"latitude":35.6644037,"longitude":139.7599456,"score":70,"chosen":true},{"index":38,"name":"上野阿美橫丁商店街 Ameyoko Shopping Street","rank":"排名第 2的Ueno觀光 (共7家)","address":"東京都台東區UenoUeno","child":0.012643587,"latitude":35.7087296,"longitude":139.7741538,"score":400,"chosen":true},{"index":17,"name":"根津神社 Nezu Shrine","rank":"排名第 2的Nezu觀光 (共11家)","address":"東京都文京區Nezu根津1-28-9113-0031","child":0.0205492008,"latitude":35.7199073,"longitude":139.7608953,"score":114,"chosen":true},{"index":243,"name":"太陽城 Sunshine City","rank":"排名第 9的豊島區觀光 (共179家)","address":"東京都豊島區1-1 Higashiikebukuro170-0013","child":0.4632720835,"latitude"

req.json() is a list of POIs each day. The order of the list means nothing.   

In [2]:
print(day2['name'])
print(day2['rank'])
print(day2['latitude'])
print(day2['longitude'])
# req.json() 

0                        Nittere Plaza
1    上野阿美橫丁商店街 Ameyoko Shopping Street
2                     根津神社 Nezu Shrine
3                    太陽城 Sunshine City
Name: name, dtype: object
0    排名第 50的港區觀光 (共626家)
1     排名第 2的Ueno觀光 (共7家)
2    排名第 2的Nezu觀光 (共11家)
3    排名第 9的豊島區觀光 (共179家)
Name: rank, dtype: object
0    35.664404
1    35.708730
2    35.719907
3    35.729950
Name: latitude, dtype: float64
0    139.759946
1    139.774154
2    139.760895
3    139.713237
Name: longitude, dtype: float64


In [3]:
with open('../apiKey.txt') as f:
    api_key = f.readline()
    f.close
gmaps.configure(api_key=api_key)

In [29]:
import json
temp = []
for index, row in enumerate(zip(day2['name'],day2['latitude'],day2['longitude'])):
#     print(i, j, k)
    data={
        "name": row[0],
        "latitude": row[1],
        "longitude": row[2]
    }
    temp.append(data)
print(json.dumps(temp, indent = 2))

[
  {
    "name": " Nittere Plaza",
    "latitude": 35.6644037,
    "longitude": 139.7599456
  },
  {
    "name": "\u4e0a\u91ce\u963f\u7f8e\u6a6b\u4e01\u5546\u5e97\u8857 Ameyoko Shopping Street",
    "latitude": 35.7087296,
    "longitude": 139.7741538
  },
  {
    "name": "\u6839\u6d25\u795e\u793e Nezu Shrine",
    "latitude": 35.7199073,
    "longitude": 139.7608953
  },
  {
    "name": "\u592a\u967d\u57ce Sunshine City",
    "latitude": 35.7299496,
    "longitude": 139.7132372
  }
]


In [30]:
temp[0]['name']

' Nittere Plaza'

In [31]:
gm = googlemaps.Client(key=api_key)
def calDist(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist
remu_hotel=(35.697795, 139.773679)
#1st spot
G = nx.MultiGraph()
elist = [('remu_hotel',temp[0]['name'],calDist(remu_hotel[0],remu_hotel[1],temp[0]['latitude'],temp[0]['longitude'])),
         ('remu_hotel',temp[1]['name'],calDist(remu_hotel[0],remu_hotel[1],temp[1]['latitude'],temp[1]['longitude'])),
         ('remu_hotel',temp[2]['name'],calDist(remu_hotel[0],remu_hotel[1],temp[2]['latitude'],temp[2]['longitude'])),
         ('remu_hotel',temp[3]['name'],calDist(remu_hotel[0],remu_hotel[1],temp[3]['latitude'],temp[3]['longitude'])),
         (temp[0]['name'],temp[1]['name'],calDist(temp[0]['latitude'],temp[0]['longitude'],temp[1]['latitude'],temp[1]['longitude'])),
         (temp[0]['name'],temp[2]['name'],calDist(temp[0]['latitude'],temp[0]['longitude'],temp[2]['latitude'],temp[2]['longitude'])),
         (temp[0]['name'],temp[3]['name'],calDist(temp[0]['latitude'],temp[0]['longitude'],temp[3]['latitude'],temp[3]['longitude'])),
         (temp[1]['name'],temp[2]['name'],calDist(temp[1]['latitude'],temp[1]['longitude'],temp[2]['latitude'],temp[2]['longitude'])),
         (temp[1]['name'],temp[3]['name'],calDist(temp[1]['latitude'],temp[1]['longitude'],temp[3]['latitude'],temp[3]['longitude'])),
         (temp[2]['name'],temp[3]['name'],calDist(temp[2]['latitude'],temp[2]['longitude'],temp[3]['latitude'],temp[3]['longitude']))
        ]
G.add_weighted_edges_from(elist)
route2=['remu_hotel']
spot1=list(dict(nx.all_pairs_dijkstra(G))['remu_hotel'][0])[1]
route2.append(spot1)
spot2=list(dict(nx.all_pairs_dijkstra(G))[spot1][0])[2]
route2.append(spot2)
spot3=list(dict(nx.all_pairs_dijkstra(G))[spot2][0])[3]
route2.append(spot3)
spot4=list(dict(nx.all_pairs_dijkstra(G))[spot3][0])[4]
route2.append(spot4)

spot1ll=(float(day2.loc[day2['name']==spot1]['latitude']),float(day2.loc[day2['name']==spot1]['longitude']))
spot1ll
spot2ll=(float(day2.loc[day2['name']==spot2]['latitude']),float(day2.loc[day2['name']==spot2]['longitude']))
spot2ll
spot3ll=(float(day2.loc[day2['name']==spot3]['latitude']),float(day2.loc[day2['name']==spot3]['longitude']))
spot3ll
spot4ll=(float(day2.loc[day2['name']==spot4]['latitude']),float(day2.loc[day2['name']==spot4]['longitude']))
spot4ll
waypoints=[spot1ll,spot2ll,spot3ll,spot4ll]
d2optRt=gm.directions(remu_hotel,
                      remu_hotel,
                      waypoints=waypoints,
                      optimize_waypoints=True)
d2optRt
#print(gm.directions(remu_hotel,
#                    spot1ll,
#                    mode="transit"))


#fig = gmaps.figure()
#marker_layer = gmaps.marker_layer(
#    locations=[spot1ll,spot2ll,spot3ll,spot4ll],
#    hover_text=route2,
#    label = route2
#)
#fig.add_layer(marker_layer)
#fig
#embed_minimal_html('user_Idday2routeTstamp.html', views=[fig])

[{'bounds': {'northeast': {'lat': 35.7323419, 'lng': 139.7770588},
   'southwest': {'lat': 35.6642844, 'lng': 139.7127647}},
  'copyrights': 'Map data ©2019',
  'legs': [{'distance': {'text': '5.0 km', 'value': 5022},
    'duration': {'text': '14 mins', 'value': 833},
    'end_address': '1 Chome-6-1 Higashishinbashi, Minato City, Tokyo 105-0021, Japan',
    'end_location': {'lat': 35.6646305, 'lng': 139.7595253},
    'start_address': '1-chōme-6 Kanda Sakumachō, Chiyoda City, Tōkyō-to 101-0025, Japan',
    'start_location': {'lat': 35.69774460000001, 'lng': 139.7739095},
    'steps': [{'distance': {'text': '0.2 km', 'value': 193},
      'duration': {'text': '1 min', 'value': 71},
      'end_location': {'lat': 35.6994501, 'lng': 139.7743042},
      'html_instructions': 'Head <b>north</b>',
      'polyline': {'points': '{e{xE}qbtYSEGC[GMCAAa@Ga@IA?GC]ESC[E[ECAI?m@EYA'},
      'start_location': {'lat': 35.69774460000001, 'lng': 139.7739095},
      'travel_mode': 'DRIVING'},
     {'distance

In [28]:
order = d2optRt[0]['waypoint_order']#list
wpOrdered=[]
for i in order:
    wpOrdered.append(waypoints[i])
print(order,waypoints,wpOrdered,sep='\n')

[3, 2, 1, 0]
[(35.7087296, 139.7741538), (35.7199073, 139.7608953), (35.7299496, 139.7132372), (35.6644037, 139.7599456)]
[(35.6644037, 139.7599456), (35.7299496, 139.7132372), (35.7199073, 139.7608953), (35.7087296, 139.7741538)]


In [ ]:
spot1ll=(float(day2.loc[day2['name']==spot1]['latitude']),float(day2.loc[day2['name']==spot1]['longitude']))
spot1ll